In [1]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv('train.csv')
df.head()

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031


In [2]:
df.isnull().sum()

id                                  0
Podcast_Name                        0
Episode_Title                       0
Episode_Length_minutes          87093
Genre                               0
Host_Popularity_percentage          0
Publication_Day                     0
Publication_Time                    0
Guest_Popularity_percentage    146030
Number_of_Ads                       1
Episode_Sentiment                   0
Listening_Time_minutes              0
dtype: int64

In [3]:
df = df[df['Episode_Length_minutes'].notnull()]
(df.isnull().sum() / len(df) * 100).sort_values(ascending=False)

Guest_Popularity_percentage    18.684069
Number_of_Ads                   0.000151
id                              0.000000
Podcast_Name                    0.000000
Episode_Length_minutes          0.000000
Episode_Title                   0.000000
Genre                           0.000000
Host_Popularity_percentage      0.000000
Publication_Time                0.000000
Publication_Day                 0.000000
Episode_Sentiment               0.000000
Listening_Time_minutes          0.000000
dtype: float64

In [4]:
df['Podcast_Episode'] = df['Podcast_Name'] + ' - ' + df['Episode_Title']
df['Podcast_Episode'].unique()
df = df.drop('id', axis=1)

df_test = df.drop('Guest_Popularity_percentage', axis=1)
df_test = df_test.drop(['Podcast_Name', 'Episode_Title'], axis=1)

<class 'pandas.core.frame.DataFrame'>
Index: 662907 entries, 1 to 749999
Data columns (total 9 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Episode_Length_minutes      662907 non-null  float64
 1   Genre                       662907 non-null  object 
 2   Host_Popularity_percentage  662907 non-null  float64
 3   Publication_Day             662907 non-null  object 
 4   Publication_Time            662907 non-null  object 
 5   Number_of_Ads               662906 non-null  float64
 6   Episode_Sentiment           662907 non-null  object 
 7   Listening_Time_minutes      662907 non-null  float64
 8   Podcast_Episode             662907 non-null  object 
dtypes: float64(4), object(5)
memory usage: 50.6+ MB


split

In [142]:
y = df_test['Listening_Time_minutes']
X = df_test.drop('Listening_Time_minutes', axis=1)

In [143]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [144]:
categorical_cols = [
    'Podcast_Episode', 'Publication_Time', 'Genre',
    'Publication_Day', 'Episode_Sentiment'
]

for col in categorical_cols:
    X_train[col] = X_train[col].astype('category')
    X_test[col] = X_test[col].astype('category')


In [145]:
X_train['Number_of_Ads'] = X_train['Number_of_Ads'].fillna(X_train['Number_of_Ads'].median())
X_test['Number_of_Ads'] = X_test['Number_of_Ads'].fillna(X_test['Number_of_Ads'].median())

In [146]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 530325 entries, 25691 to 137953
Data columns (total 8 columns):
 #   Column                      Non-Null Count   Dtype   
---  ------                      --------------   -----   
 0   Episode_Length_minutes      530325 non-null  float64 
 1   Genre                       530325 non-null  category
 2   Host_Popularity_percentage  530325 non-null  float64 
 3   Publication_Day             530325 non-null  category
 4   Publication_Time            530325 non-null  category
 5   Number_of_Ads               530325 non-null  float64 
 6   Episode_Sentiment           530325 non-null  category
 7   Podcast_Episode             530325 non-null  category
dtypes: category(5), float64(3)
memory usage: 19.4 MB


In [ ]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np

# Define RMSE scorer
rmse_scorer = make_scorer(
    lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)),
    greater_is_better=False
)

# Define model
model = LGBMRegressor(random_state=42)

# Hyperparameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'num_leaves': [20, 31, 40]
}

# Grid search
grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring=rmse_scorer,
    cv=3,
    verbose=1,
    n_jobs=-1
)

grid.fit(X_train, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


In [137]:
print("rmse_scorer:", -grid.best_score_)  # convert back to positive RMSE
print("Best Parameters:", grid.best_params_)

rmse_scorer: 10.468236600466556
Best Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'num_leaves': 40}


In [123]:
from lightgbm import LGBMRegressor

model = LGBMRegressor()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004779 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5223
[LightGBM] [Info] Number of data points in the train set: 530325, number of used features: 8
[LightGBM] [Info] Start training from score 45.751311
